## Part 1: Preprocessing

#### Step 1: Import the data and view the first five rows

In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


#### Step 2: Determine the number of unique values in each column & identify y labels

In [30]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [31]:
# Explore the classes for Attrition Column
attrition_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

- Attrition is Binary and Mutually Exclusive
- Sigmoid is best option
- Encoding: Label Encoding or OneHot Encoding

In [32]:
# Explore the classes for Department Column
attrition_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

- Department is Multi-class and Mutually Exclusive
- Softmax is best option
- Encoding: OneHot Encoding

#### Step 3: Create y_df with identified column(s)

In [33]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


#### Step 4: Create X_df with selected features

In [34]:
# Create a list of at least 10 column names to use as X data
X_columns = ["Education", "Age", "DistanceFromHome", "JobSatisfaction", "OverTime", "StockOptionLevel",
              "WorkLifeBalance",  "YearsAtCompany", "YearsSinceLastPromotion", "NumCompaniesWorked"]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [35]:
# Explore categorical column
X_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

#### Step 5: Split the data into training and testing sets

In [36]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

#### Step 6: Convert X data to numeric data types
- In this step a generic OneHotEncoding method is applied to encode all categorical columns

In [37]:
# Convert your X data to numeric data types however you see fit
# Encode categorical variables in X_train and X_test
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Align columns in training and testing sets
X_train, X_test = X_train.align(X_test, join="inner", axis=1)

#### Step 7: Scale the X data

In [38]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Step 8.1: Encode y data - Department

In [39]:
# Import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data for the Department column
department_encoder.fit(y_train[["Department"]])

# Apply the encoder to the training and testing data
y_train_department = department_encoder.transform(y_train[["Department"]])
y_test_department = department_encoder.transform(y_test[["Department"]])

# Display the encoded output for the Department column
print("Encoded Department Training Data:")
print(y_train_department)

Encoded Department Training Data:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


#### Step 8.2: Encode y data - Attrition

In [40]:
# Create a OneHotEncoder instance for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data for the Attrition column
attrition_encoder.fit(y_train[["Attrition"]])

# Apply the encoder to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[["Attrition"]])
y_test_attrition = attrition_encoder.transform(y_test[["Attrition"]])

# Display the encoded output for the Attrition column
print("Encoded Attrition Training Data:")
print(y_train_attrition)

Encoded Attrition Training Data:
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [41]:
# Find the number of columns in the X training data
# input_features = X_train_scaled.shape[1]

# Create the input layer
# input_layer = layers.Input(shape=(input_features,), name="input")

# in this code we have the shapr of input as 11 vs 10 because onehot encoding created one more column
input_layer = layers.Input(shape=(11,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [42]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=32, activation="relu", name="department_hidden")(shared_layer2)

# Create the output layer
department_output = layers.Dense(units=3, activation="softmax", name="department_output")(department_hidden)


In [43]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation="relu", name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation="sigmoid", name="attrition_output")(attrition_hidden)


In [44]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │        768 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      8,320 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ dense_5[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ dense_5[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# Exploration Step

print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)
print("Number of features in X_train:", len(X_train.columns))

X_train columns: Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked', 'OverTime_No',
       'OverTime_Yes'],
      dtype='object')
X_test columns: Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked', 'OverTime_No',
       'OverTime_Yes'],
      dtype='object')
Number of features in X_train: 11


In [46]:
# Train the model
model.fit(
    X_train_scaled, 
    {
        "department_output": y_train_department,
        "attrition_output": y_train_attrition
    },
    validation_data=(
        X_test_scaled, 
        {
            "department_output": y_test_department,
            "attrition_output": y_test_attrition
        }
    ),
    epochs=100,
    batch_size=32
)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - attrition_output_accuracy: 0.7745 - attrition_output_loss: 0.5988 - department_output_accuracy: 0.6219 - department_output_loss: 0.9129 - loss: 1.5120 - val_attrition_output_accuracy: 0.8152 - val_attrition_output_loss: 0.4650 - val_department_output_accuracy: 0.6413 - val_department_output_loss: 0.7641 - val_loss: 1.2455
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8476 - attrition_output_loss: 0.4175 - department_output_accuracy: 0.6729 - department_output_loss: 0.7578 - loss: 1.1753 - val_attrition_output_accuracy: 0.8152 - val_attrition_output_loss: 0.4331 - val_department_output_accuracy: 0.6413 - val_department_output_loss: 0.7749 - val_loss: 1.2225
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8424 - attrition_output_loss: 0.3990 - department_output_accuracy: 0.6539 - department_output_loss: 0.7777 - loss: 1.1765 - val_attrition_output_accuracy: 0.8152 - 

In [47]:
# Evaluate the model with the testing data
evaluation_results = model.evaluate(
    X_test_scaled,
    {
        "department_output": y_test_department,
        "attrition_output": y_test_attrition
    }
)

# Print the evaluation results
print(f"Loss: {evaluation_results[0]}")
print(f"Department Output Loss: {evaluation_results[1]}")
print(f"Attrition Output Loss: {evaluation_results[2]}")
print(f"Department Output Accuracy: {evaluation_results[3]}")
print(f"Attrition Output Accuracy: {evaluation_results[4]}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7801 - attrition_output_loss: 1.4217 - department_output_accuracy: 0.5551 - department_output_loss: 2.9920 - loss: 4.4324 
Loss: 4.279829025268555
Department Output Loss: 2.8123557567596436
Attrition Output Loss: 1.345543384552002
Department Output Accuracy: 0.7989130616188049
Attrition Output Accuracy: 0.5570651888847351


In [48]:
# Print the accuracy for both department and attrition
department_accuracy = evaluation_results[3]  # Department output accuracy
attrition_accuracy = evaluation_results[4]   # Attrition output accuracy

print(f"Department predictions accuracy: {department_accuracy}")
print(f"Attrition predictions accuracy: {attrition_accuracy}")

Department predictions accuracy: 0.7989130616188049
Attrition predictions accuracy: 0.5570651888847351


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Generally Accuracy is a good metric. In this case, as there is an imbalance in classes for both output labels, either we balance the classes or use alternative metrics like - precision, F1-score or a combination
2. Department Output - Activation Function: Softmax:: Since Department is a multi-class categorical variable with mutually exclusive options, softmax ensures the output represents probabilities across all departments, summing to 1.
3. Attrition Output: Activation Function: Sigmoid: Attrition is binary (Yes/No). A sigmoid activation function outputs probabilities between 0 and 1, making it suitable for binary classification.
4. Improvements that can be used in this are: class balance techniques; hyper-parameter tuning; try different architectures

In [49]:
#this is extra to get configuration details

#filter harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 

#you need to install watermark
from watermark import watermark 
print(watermark()) 
print(watermark(iversions=True, globals_=globals()))

Last updated: 2025-01-05T12:14:44.912745-07:00

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.25.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

sklearn   : 1.4.2
numpy     : 1.26.4
watermark : 2.5.0
sys       : 3.10.14 (main, May  6 2024, 14:42:37) [Clang 14.0.6 ]
IPython   : 8.25.0
ipywidgets: 8.1.5
tensorflow: 2.18.0
pandas    : 2.2.2
keras     : 3.7.0
xarray    : 2023.6.0
json      : 2.0.9

